In [33]:
import pandas as pd
from pathlib import Path


PROJECTDIR = Path('.').resolve().parents[2]
excelpath = f'{PROJECTDIR}/external_data/organ_development/NIHMS1530688-supplement-2.xlsx'


human = pd.read_excel(io=excelpath, sheet_name='SI Table 14', skiprows=2)
display(human[human.classification != 'Conserved'])

/home/felixl/mambaforge/envs/rodent_loss/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


,Human_ID,Mouse_ID,Rat_ID,Rabbit_ID,Opossum_ID,comparison,prob_same,on_branch,classification
5,ENSG00000001629,ENSMUSG00000040351,ENSRNOG00000005502,ENSOCUG00000004576,ENSMODG00000010686,MRRab_H,6.646957e-03,H,Human
6,ENSG00000001631,ENSMUSG00000000600,ENSRNOG00000007937,ENSOCUG00000004570,ENSMODG00000010670,MRRabH_O,9.240000e-05,NaN,Eutherian/opo
10,ENSG00000003436,ENSMUSG00000027082,ENSRNOG00000005039,ENSOCUG00000008253,ENSMODG00000010692,M_R,5.270000e-08,R,Rat
13,ENSG00000003989,ENSMUSG00000031596,ENSRNOG00000011016,ENSOCUG00000011284,ENSMODG00000019050,MRRab_H,4.652637e-03,H,Human
22,ENSG00000004897,ENSMUSG00000020687,ENSRNOG00000005904,ENSOCUG00000010815,ENSMODG00000010908,MRRabH_O,7.020000e-06,NaN,Eutherian/opo
...,...,...,...,...,...,...,...,...,...
3960,ENSG00000242247,ENSMUSG00000054277,ENSRNOG00000046472,ENSOCUG00000024633,ENSMODG00000006872,MRRabH_O,1.451971e-03,NaN,Eutherian/opo
3961,ENSG00000243943,ENSMUSG00000062761,ENSRNOG00000039815,ENSOCUG00000013611,ENSMODG00000015600,MRRabH_O,1.040000e-06,NaN,Eutherian/opo
3970,ENSG00000250305,ENSMUSG00000039620,ENSRNOG00000011253,ENSOCUG00000024898,ENSMODG00000019077,MRRab_H,1.415362e-03,none,MRRab_H_NA
3974,ENSG00000258890,ENSMUSG00000018372,ENSRNOG00000014354,ENSOCUG00000012756,ENSMODG00000003779,M_R,2.423449e-03,none,M_R_NA


## For each tissue, extract genes with rodent-specific changes in expression trajectory

In [34]:
from tqdm import tqdm

relevant_tables = {
    '14': 'brain',
    '15': 'cerebellum',
    '16': 'heart',
    '17': 'liver',
    '18': 'testis',
}


def extract_rodentspecific_trajectories(excelpath, relevant_tables):
    col = {}
    for table_index, tissue in tqdm(relevant_tables.items()):
        df = pd.read_excel(io=excelpath, sheet_name=f'SI Table {table_index}', skiprows=2)
        
        roden_specific_tracejectory = df[(df.classification == 'Rodents')]
        col[tissue] = roden_specific_tracejectory
    return col
    
    
    # print(relevant_tables)
    
    
tissue2rodent_tracectory_df = extract_rodentspecific_trajectories(excelpath, relevant_tables)

  0%|          | 0/5 [00:00<?, ?it/s]/home/felixl/mambaforge/envs/rodent_loss/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
 20%|██        | 1/5 [00:04<00:17,  4.36s/it]/home/felixl/mambaforge/envs/rodent_loss/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
 40%|████      | 2/5 [00:08<00:12,  4.22s/it]/home/felixl/mambaforge/envs/rodent_loss/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
 60%|██████    | 3/5 [00:12<00:08,  4.22s/it]/home/felixl/mambaforge/envs/rodent_loss/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.p

In [35]:
doapr_path = f'{PROJECTDIR}/milestones/data/doapr/sigdown_and_predictedTargets_tsM_tsH.tsv'


def load_filter_doapr(path, organism='human', mirna=''):
    """
    Return list of human Uniprot gene IDs
    """
    df = pd.read_csv(path, sep='\t')
    
    if organism:
        df = df[df.organism == organism]
    if mirna:
        df = df[df.mirna == mirna]
    
    # filter out genes without 1:1 orthologs (their orthomap is NaN)
    df = df[~df.orthomap.isna()]

    
    human_ids = df.orthomap.apply(lambda x: x.split('|')[0]).to_list()
    
    return human_ids
    

humantargets_mir197_mir769 = load_filter_doapr(doapr_path)
humantargets_mir197 = load_filter_doapr(doapr_path, mirna='Mir-197-3p')
humantargets_mir769 = load_filter_doapr(doapr_path, mirna='Mir-769-5p')

## Find targets in genes with rodent-specific development trajectory

M = total options
n = number positive outcomes
N = sample size
x = number of sucessfull draws

In [45]:
from scipy.stats import hypergeom


for tissue, tracetorydf in tissue2rodent_tracectory_df.items():
    # print(tissue)
    display(tracetorydf[tracetorydf.Human_ID.isin(humantargets_mir197_mir769)])
    overlap = tracetorydf[tracetorydf.Human_ID.isin(humantargets_mir197_mir769)].index.size
    
    if overlap > 0:
        M = 18000
        n = tracetorydf.index.size
        N = len(humantargets_mir197_mir769)
        x = overlap
        
        # print(x, M, n, N)
        pval = hypergeom.sf(x, M, n, N)
        
        expected_draws = (N*n)/M
        
        fold_enrichment = x / expected_draws
        
        print(f'Found {overlap} gene overlapping in {tissue}. Fishers exact test: {fold_enrichment:.2f} enrichment with p-value {pval:.1e}')
        if pval <= 0.05:
            print('This is significant')
    
    
    
    

,Human_ID,Mouse_ID,Rat_ID,Rabbit_ID,Opossum_ID,comparison,prob_same,on_branch,classification


,Human_ID,Mouse_ID,Rat_ID,Rabbit_ID,Opossum_ID,comparison,prob_same,on_branch,classification
625,ENSG00000104442,ENSMUSG00000027599,ENSRNOG00000013253,ENSOCUG00000000392,ENSMODG00000008168,MR_Rab,0.000077,MR,Rodents


Found 1 gene overlapping in cerebellum. Fishers exact test: 2.20 enrichment with p-value 7.6e-02


,Human_ID,Mouse_ID,Rat_ID,Rabbit_ID,Opossum_ID,comparison,prob_same,on_branch,classification


,Human_ID,Mouse_ID,Rat_ID,Rabbit_ID,Opossum_ID,comparison,prob_same,on_branch,classification


,Human_ID,Mouse_ID,Rat_ID,Rabbit_ID,Opossum_ID,comparison,prob_same,on_branch,classification
2345,ENSG00000165025,ENSMUSG00000021457,ENSRNOG00000012160,ENSOCUG00000017539,ENSMODG00000019263,MR_Rab,0.00001,MR,Rodents


Found 1 gene overlapping in testis. Fishers exact test: 1.69 enrichment with p-value 1.2e-01
